In [1]:
import pandas as pd
import requests 

/Users/shashanksingh/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/shashanksingh/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [3]:
fuel_df = pd.read_csv('fuel_prices.csv')
fuel_df['Date'] = pd.to_datetime(fuel_df['Date'])
start_date = fuel_df['Date'].min().strftime('%Y-%m-%d')
end_date = fuel_df['Date'].max().strftime('%Y-%m-%d')
print(f"Date range found: {start_date} to {end_date}")

Date range found: 2017-06-16 to 2025-10-22


In [4]:
CITY_COORDS = {
    'Delhi': {'lat': 28.61, 'lon': 77.21},
    'Mumbai': {'lat': 19.08, 'lon': 72.88},
    'Chennai': {'lat': 13.08, 'lon': 80.27},
    'Kolkata': {'lat': 22.57, 'lon': 88.36}
}

In [8]:
api_url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    'latitude': [city['lat'] for city in CITY_COORDS.values()],
    'longitude': [city['lon'] for city in CITY_COORDS.values()],
    'start_date': start_date,
    'end_date': end_date,
    'daily': 'temperature_2m_mean,precipitation_sum',
    'timezone': 'Asia/Kolkata' # Use local timezone
}
response = requests.get(api_url, params=params)

if response.status_code != 200:
    print(f"Error: API request failed with status code {response.status_code}")
    print(response.json())
    exit()

results = response.json()
print("API call successful. Processing data...")

all_weather_df = []
for i, city_data in enumerate(results):
    city_name = list(CITY_COORDS.keys())[i]
    daily_data = city_data['daily']
    df = pd.DataFrame(daily_data)
    df['City'] = city_name
    
    all_weather_df.append(df)
weather_df = pd.concat(all_weather_df, ignore_index=True)

API call successful. Processing data...


In [10]:
weather_df.rename(columns={
    'time': 'Date',
    'temperature_2m_mean': 'Temp_Mean_C',
    'precipitation_sum': 'Precipitation_mm'
}, inplace=True)
weather_df['Date'] = pd.to_datetime(weather_df['Date'])
weather_df = weather_df[['Date', 'City', 'Temp_Mean_C', 'Precipitation_mm']]

In [11]:
weather_df.to_csv("Weather Data.csv", index=False)

In [12]:
weather_df.head()

,Date,City,Temp_Mean_C,Precipitation_mm
0,2017-06-16,Delhi,31.1,0.4
1,2017-06-17,Delhi,30.6,7.0
2,2017-06-18,Delhi,30.9,3.1
3,2017-06-19,Delhi,31.5,5.5
4,2017-06-20,Delhi,29.8,10.2
